In [1]:
%cd /content/drive/My Drive/Colab Notebooks/regaetton_songs_nlp

/content/drive/My Drive/Colab Notebooks/regaetton_songs_nlp


In [2]:
!pip install snorkel

     |████████████████████████████████| 153kB 6.7MB/s 
     |████████████████████████████████| 6.7MB 18.7MB/s 
     |████████████████████████████████| 1.8MB 58.7MB/s 
     |████████████████████████████████| 3.8MB 47.4MB/s 
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556408 sha256=78338f88ad04e87c69585f9fbaf4375b9e1c06c0fdd357a548507fff0e0f348e
  Stored in directory: /root/.cache/pip/wheels/de/63/64/3699be2a9d0ccdb37c7f16329acf3863fd76eda58c39c737af
Successfully built networkx
ERROR: tensorflow 2.4.0 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: networkx 2.5
    Uninstalling networkx-2.5:
      Successful

In [3]:
import pandas as pd
import re

from snorkel.analysis import get_label_buckets
from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis

In [52]:
songs_path = '/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp/data/updated_lyrics.csv'
final_train_data_path = '/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp/data/train_lyrics.csv'
final_eval_data_path = '/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp/data/eval_lyrics.csv'

In [5]:
songs = pd.read_csv(songs_path)
songs.head()

,song_name,artist,lyrics,sexual_content,women_denigration,drugs
0,Te quedas o te vas,Nicky Jam,Ya ni sé lo que te está pasando\nSiento que co...,0,0,0
1,Sola,Anuel,Real hasta la muerte baby\nWelcome to the remi...,1,1,1
2,Pensando en ti. Feat Wisin,Anuel,bebé Real Hasta La muerte baby\nW en conexión ...,1,1,0
3,Desperte sin ti,Noriel,Desperté sin ti\nLa casa se me vino encima\nDi...,0,0,0
4,Tú y yo,Makano,Tú y yo Un ramo de imagines\ntú y yo Una simpl...,0,0,0


Get explicit sexual content songs that were already label to identify some patterns that we can then use for weak supervision.

In [6]:
def show_lyrics(df, index):
    """
    Helper function to read lyrics
    """
    return print(df.lyrics.iloc[index].strip())

def change_accent_mark(lyrics):
    """
    Change á, é, í, ó, ú for a, e, i, o, u.
    """
    import re

    tildes = ['á', 'é', 'í', 'ó', 'ú']
    for tilde in tildes:
        if tilde == 'á':
            replace = 'a'
        elif tilde == 'é':
            replace = 'e'
        elif tilde == 'í':
            replace = 'i'
        elif tilde == 'ó':
            replace = 'o'
        elif tilde == 'ú':
            replace = 'u'
        
        lyrics = re.sub(tilde, replace, lyrics)

    return lyrics



# This are rules for explicit sexual content
- comer, comerte,comerme,  
- desnuda, 
- encima de mi/ti, 
- adentro de ti/mi,
- en el sexo,
- hacer/hacemos/ haciendo el amor
-  chingar/chingas/ chingando,
- mi cama, en su cama, terminar en la cama,  la cama,
- tocando/ tocaba/tocar/ tocarte, 
- tu en mi habitacion,
- coger, cogerte/cojerte, 
- recorriendo todo tu cuerpo/ recorriendo/recorrer tu cuerpo, 
- te coma/te comi
- culo
- culea,
- te/se lo/la meto/toco/ metia/entierro
- quiero tenerte,  quiero darte, 
- mamar
- nos matamos/matabamos /comimos/ devoramos/ cogimos/revolcamos/
- hacerte/haciendote mia
- tetas
- en cuatro
- haciendolo/hacertelo metertelo, hacerlo
- nalgas
- sin ropa
- desvestir/desvestirse/
- polvo
- condon
- penetrar
- meterlo/metiendolo /metertelo
- chupar
- sudar
- orgasmo
- devorar

# No sexual content

- 

In [7]:
songs_copy = songs.copy()

# filter wrong lyrics
songs_copy['dtype'] = songs_copy.lyrics.apply(lambda x: 1 if type(x) == str else 2)
songs_copy = songs_copy[songs_copy.dtype == 1]
songs_copy.reset_index(drop=True, inplace=True)
songs_copy['lyrics'] = songs_copy.lyrics.apply(change_accent_mark)

In [8]:
SEXUAL_CONTENT = 1
NO_SC = 0
ABSTAIN = -1
@labeling_function()
def lb_comer(df):
    """
    search for patterns like: comer, comerte, comerme, 
    comernos, 
    """
    pattern = r'comer\w*'
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_desnuda(df):
    """
    search for patterns like: desnudar, desnuda, desnudas
    desnudate, desnudo, etc.
    """
    pattern = r'desnud\w*'
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_encima_de(df):
    """
    search for patterns like: encima de mi/ti/la
    """
    pattern = r'encima de [tml]i'
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_adentro_de(df):
    """
    search for patterns like: adentro de mi/ti
    """
    pattern = r'adentro de \w*'
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_sexo(df):
    pattern = r"sexo"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_hacer_amor(df):
    """
    search for patterns like: hacer/haciendo, hacemos
    el amor
    """
    pattern = r"hac\w* el a\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_chingar(df):
    """
    search for patterns like: chingar, chingamos, chingando etc.
    """
    pattern = r"ching\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_cama(df):
    """
    search for patterns like: tu/mi/la/ cama
    """
    pattern = r"[a-z]\w* cama"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_tocar(df):
    """
    search for patterns like: tocar, tocarte, tocando
    tocandonos, etc.
    """
    pattern = r"toca\w+"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_habitacion(df):
    """
    Search for patterns like: la/mi/tu habitacion
    """
    pattern = r"[a-z]\w* habitacion"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_coger(df):
    """
    search for patterns like: coger, cogerte, cogerla, etc.
    We use [gj] for misspelling
    """
    pattern = r"co[gj]er\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_recorrer(df):
    """
    search for patterns like: recorrer/recorriendo tu cuerpo
    """
    pattern = r"recorr\w* \w* cuerpo"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_te_comi(df):
    """
    Search for patterns like: te/me/nos como/comi/comia
    """
    pattern = r"[tm][ei] com[oei]"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_culo(df):
    """
    search for patterns like: culo, culote, culona etc.
    """
    pattern = r"culo\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_culea(df):
    """
    Search for patterns like: culea, culear, culeando, etc.
    """
    pattern = r"culea\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_meto_entierro_toco(df):
    """
    search for patterns like: se/te lo/la meto/entierro/toco
    """
    pattern = r"(te|se) (lo|la) (met\w*|toco|entierro)"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_quiero_darte_tocarte(df):
    """
    search for patterns like: quiero tocarte/darte
    """
    pattern = r"(quier tocarte|quiero darte)"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_mamar(df):
    pattern = r'mamar'
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_nos_variants(df):
    """
    Search for patterns like: nos matamos/matabamos/comimos/devoramos/cogimos/revolcamos
    """
    pattern = r"(nos) (mata\w*os|com\w*os|devor\w*os|cog\w*os|revol\w*os)"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_hacer_mia(df):
    """
    Search for patterns like: hacerte/haciendote/ mia
    """
    pattern = r"(hacer\w*|ser\w*) (mia)"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_tetas(df):
    pattern = r"tetas"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_en_cuatro(df):
    pattern = r"en cuatro"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_hacerlo(df):
    """
    Search for patterns like: hacertelo/hacerlo/metertelo/tocalo
    sientelo, etc
    """
    pattern = r"(hac\w*lo|meter\wlo|toc\w*|sientelo)"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_nalgas(df):
    pattern = r"nalga\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN


@labeling_function()
def lb_sin_ropa(df):
    pattern = r"sin ropa"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_desvestir(df):
    """
    Search for patterns like: desvestir, desvestidos, desvestirte, etc
    """
    pattern = r"desvest\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_polvo(df):
    pattern = r"polvo\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_condon(df):
    pattern = r'condon\w*'
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_penetrar(df):
    """
    Search for patterns like: penetrar, penetracion, penetrarte, etc
    """
    pattern = r"penetr\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_meter(df):
    """
    Search for patterns like:meterlo/metertelo
    """
    pattern = r"meter\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_chupar(df):
    pattern = r"chupar\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

@labeling_function()
def lb_devorar(df):
    pattern = r"devorar\w*"
    return SEXUAL_CONTENT if re.search(pattern, df.lyrics.lower()) else ABSTAIN

In [9]:
labeling_functions = [lb_adentro_de, lb_cama, lb_chingar, lb_chupar, lb_coger, lb_comer,
                      lb_condon, lb_culea, lb_culo, lb_desnuda, lb_desvestir, lb_devorar, 
                      lb_en_cuatro, lb_encima_de, lb_habitacion, lb_hacer_amor, lb_hacer_mia, lb_hacerlo,
                      lb_mamar, lb_meter, lb_meto_entierro_toco, lb_nalgas, lb_nos_variants,
                      lb_penetrar, lb_polvo, lb_quiero_darte_tocarte, lb_recorrer, lb_sexo, 
                      lb_sin_ropa, lb_te_comi, lb_tetas, lb_tocar]

# Split the data

- The evaluation set is going the be the 600 hand-labeled lyrics
- The rest of the lyrics is going to be for training

In [10]:
eval_data = songs_copy[:600].copy()

# Joining the explicit and implicit labels to just sexual content. So, 1 means the lyrics have
# some sexual content and 0 means they don't have any sexual content
eval_data.sexual_content.replace({2:1}, inplace=True)
train_data = songs_copy[600:].copy()
train_data.reset_index(drop=True, inplace=True)

In [11]:
applier = PandasLFApplier(lfs=labeling_functions)


In [12]:
l_train = applier.apply(df=train_data)

100%|██████████| 8065/8065 [00:05<00:00, 1448.05it/s]


# From snorkel docs

- Polarity: The set of unique labels this LF outputs (excluding abstains)

- Coverage: The fraction of the dataset the LF labels

- Overlaps: The fraction of the dataset where this LF and at least one other LF label 
- Conflicts: The fraction of the dataset where this LF and at least one other LF label and disagree
- Correct: The number of data points this LF labels correctly (if gold labels are provided)
- Incorrect: The number of data points this LF labels incorrectly (if gold labels are provided)
- Empirical Accuracy: The empirical accuracy of this LF (if gold labels are provided)


In [13]:
LFAnalysis(L=l_train, lfs=labeling_functions).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lb_adentro_de,0,[1],0.007812,0.006820,0.0
lb_cama,1,[1],0.175077,0.125108,0.0
lb_chingar,2,[1],0.024055,0.022071,0.0
lb_chupar,3,[1],0.006200,0.005084,0.0
lb_coger,4,[1],0.028890,0.019839,0.0
lb_comer,5,[1],0.073652,0.059392,0.0
lb_condon,6,[1],0.012027,0.011779,0.0
lb_culea,7,[1],0.003224,0.002356,0.0
lb_culo,8,[1],0.052697,0.039802,0.0
lb_desnuda,9,[1],0.041414,0.034594,0.0


In [14]:
lb_train_df = pd.DataFrame(l_train)
lb_train_df['count'] = lb_train_df.sum(axis=1)

# Getting the labels

In the lb_train_df, we have the lyrics index as index and the columns are the different rules that we created using snorkel. So, if we sum all the row values we will be able to know how many of the rules each lyrics has. For example, a lyric with the sum value of -32 means that it does not have any of the rules, a sum value of -31 means that the lyric has 1 rule and so on.

To know if a song has sexual content on the lyrics, we will assume that if the song has 2 or more of the rules it has sexual content and we will set the value to 1 otherwise it will be 0 which means no sexual content.

There are many things that we can start to consider here, for example, that some of the rules could be more noisy than others, so we can give more or less weight to some of them.

We can start training different classifiers with these parameters and come back here to try other set up.



In [15]:
lb_train_df['count'].value_counts()

-32    3243
-30    2045
-28    1351
-26     731
-24     372
-22     161
-20      83
-18      36
-16      17
-14      11
-12       8
-8        3
-10       2
-6        2
Name: count, dtype: int64

In [30]:
# Cast to absolute value to better understanding
threshold = 30
lb_train_df['label'] = lb_train_df['count'].apply(lambda x: 1 if abs(x) < threshold else 0)
lb_train_df.head() 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,count,label
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-32,0
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-32,0
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-24,1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-32,0
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-32,0


In [31]:
# Now add the label column from lb_train_df to the train_data
train_data['sexual_content'] = lb_train_df.label
train_data.head()

,song_name,artist,lyrics,sexual_content,women_denigration,drugs,dtype
0,NaN,La Base,che vos la corres de derretir\ny a mi que me i...,0,-1,-1,1
1,NaN,Eddy Lover,\n\nVete si ya tu no me quieres\nya no sigo ma...,0,-1,-1,1
2,NaN,Tego Calderon,Es Tego calder con el voltaje\n\n\n\n\n\nCon L...,1,-1,-1,1
3,NaN,Hector El Father,Ladies and gentlemen\nAnd now from Puerto Rico...,0,-1,-1,1
4,NaN,Alexis Y Fido,\n\nEsta es la ultima que me juego!!!\nSolo un...,0,-1,-1,1


In [32]:
train_data.sexual_content.value_counts()

0    5288
1    2777
Name: sexual_content, dtype: int64

In [33]:
sc = train_data[train_data.sexual_content == 1]
no_sc = train_data[train_data.sexual_content == 0]

In [36]:
# show_lyrics(sc, 65)

In [40]:
l_eval = applier.apply(df=eval_data)
lb_eval_df = pd.DataFrame(l_eval)
lb_eval_df['count'] = lb_eval_df.sum(axis=1)
lb_eval_df['count'].value_counts()


100%|██████████| 600/600 [00:00<00:00, 1467.94it/s]


-32    221
-30    153
-28     98
-26     71
-24     24
-22     20
-18      5
-20      4
-16      2
-14      1
-8       1
Name: count, dtype: int64

In [42]:
eval_data.sexual_content.value_counts()

0    310
1    290
Name: sexual_content, dtype: int64

Evaluating our labeling functions, we can see that from the 600 hand-labeled lyrics we have 290 sexual content lyrics, meanwhile, the labeling functions get a total of 224 sexual content lyrics assuming that the lyrics accomplish at least 3 of the labeling rules. This is a 77%. 

# Save the train and eval sets

We create the labeling functions using a dataset without the accent marks in the lyrics, but our final sets are going to be the ones with the accent marks. In some cases, accent marks can be really important to determine meaning, so if we find that this is causing us problems we should come back and run the labeling functions again but this time taking into account the accent marks. 

One of the reasons for not doing it with accent marks at the first place is the fact that the lyrics were written by multiple users and not all of them uploaded the lyrics with the correct grammar and the correct punctuation.

In [43]:
final_eval_data = songs[:600].copy()

# Joining the explicit and implicit labels to just sexual content. So, 1 means the lyrics have
# some sexual content and 0 means they don't have any sexual content
final_eval_data.sexual_content.replace({2:1}, inplace=True)
final_train_data = songs[600:].copy()
final_train_data.reset_index(drop=True, inplace=True)

In [45]:
# Making sure that the datasets have the same order

print("EVALUATION DATA")
print(final_eval_data.tail(1))
print(eval_data.tail(1))
print('TRAINING DATA')
print(final_train_data.tail(1))
print(train_data.tail(1))

EVALUATION DATA
                     song_name     artist  ... women_denigration  drugs
599  Guerrera part dellafuente  C Tangana  ...                 0      0

[1 rows x 6 columns]
                     song_name     artist  ... drugs  dtype
599  Guerrera part dellafuente  C Tangana  ...     0      1

[1 rows x 7 columns]
TRAINING DATA
        song_name    artist  ... women_denigration  drugs
8064  El desorden  Don Omar  ...                -1     -1

[1 rows x 6 columns]
        song_name    artist  ... drugs  dtype
8064  El desorden  Don Omar  ...    -1      1

[1 rows x 7 columns]


In [51]:
# Now that we know they are the same, we just pass the sexual_content column from 
# train_data to final_train_data and save the datasets

final_train_data['sexual_content'] = train_data['sexual_content']
final_train_data.head()

,song_name,artist,lyrics,sexual_content,women_denigration,drugs
0,NaN,La Base,che vos la corres de derretir\ny a mi que me i...,0,-1,-1
1,NaN,Eddy Lover,\n\nVete si ya tu no me quieres\nya no sigo ma...,0,-1,-1
2,NaN,Tego Calderon,Es Tego calder con el voltaje\n\n\n\n\n\nCon L...,1,-1,-1
3,NaN,Hector El Father,Ladies and gentlemen\nAnd now from Puerto Rico...,0,-1,-1
4,NaN,Alexis Y Fido,\n\nEsta es la última que me juego!!!\nSolo un...,0,-1,-1


In [54]:
final_train_data.to_csv(final_train_data_path, index=False)
final_eval_data.to_csv(final_eval_data_path, index=False)